In [1]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import datetime as dt
import datetime as dt
import pandas as pd
import numpy as np

## Liquity 

In [16]:
sample_transport=RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/liquity/liquity',
    verify=True,
    retries=5,
)
client = Client(
    transport=sample_transport
)


query = gql('''
{
  liquidations(orderBy: liquidatedCollateral, orderDirection: desc, first: 10) {
    liquidator {
      id
    }
    transaction {
      timestamp
    }
    liquidatedCollateral
    liquidatedDebt
    }
}''')

In [17]:
response = client.execute(query)

In [53]:
Liq_collateral = []
Liq_debt = []
date = []

for i in range(len(response['liquidations'])):
    Liq_collateral.append (np.round(float(response['liquidations'][i]['liquidatedCollateral']), 2))
    Liq_debt.append (np.round(float((response['liquidations'][i]['liquidatedDebt'])), 2))
    date.append (dt.datetime.fromtimestamp(response['liquidations'][i]['transaction']['timestamp']))

In [122]:
df = pd.DataFrame(index = ["Time", "Liquidated ETH", "Liquidated Debt"], data = [date, Liq_collateral, Liq_debt]).T
df["ETH Liquidation Price"] = df['Liquidated Debt'] / df['Liquidated ETH']
df["Current ETH price"] = 2700
profit = ((df["Current ETH price"] - df["ETH Liquidation Price"] ) * df["Liquidated ETH"]) / 1000000

rounded_profit = []
for i in range(len(profit)):
    rounded_profit.append(round(profit[i], 2))
    

df["Stability Pool Profit in Millions $"] = rounded_profit

In [128]:
df.iloc[:, [0, 1, 3, 4, 5]]

,Time,Liquidated ETH,ETH Liquidation Price,Current ETH price,Stability Pool Profit in Millions $
0,2021-05-19 20:57:47,16730.4,1990.41,2700,11.87
1,2021-05-19 21:16:14,15501.3,1805.88,2700,13.86
2,2021-05-19 21:15:39,9253,1858.77,2700,7.78
3,2021-05-19 20:54:35,3847.18,2127.27,2700,2.20
4,2021-05-19 20:54:51,1362.98,2066.43,2700,0.86
5,2021-05-19 20:52:41,638.86,2215.84,2700,0.31
6,2021-05-19 20:59:02,442.09,1942.22,2700,0.34
7,2021-04-18 11:40:13,354.44,1866.54,2700,0.30
8,2021-04-08 03:18:27,103.38,1788.29,2700,0.09
9,2021-04-06 08:25:38,99.5,1937.69,2700,0.08


In [129]:
query = gql('''
{
  redemptions(orderBy: collateralRedeemed, orderDirection: desc, first: 100) {
    redeemer {
      id
    }
    transaction {
      timestamp
    }
    collateralRedeemed
    }
}''')

In [130]:
response = client.execute(query)

In [136]:
Redeemed_collateral = []
redeemer = []
date = []

for i in range(len(response['redemptions'])):
    Redeemed_collateral.append (np.round(float(response['redemptions'][i]['collateralRedeemed']), 2))
    redeemer.append (response['redemptions'][i]['redeemer']['id'])
    date.append (dt.datetime.fromtimestamp(response['redemptions'][i]['transaction']['timestamp']))

In [140]:
df = pd.DataFrame(index = ["Time", "Redeemed ETH", "Redeemer"], data = [date, Redeemed_collateral, redeemer]).T
df.index = df["Time"]
df = df.iloc[:, 1:]
df = df[(df.index.day == 19) * (df.index.month == 5)]

In [172]:
df

,Redeemed ETH,Redeemer
Time,,
2021-05-19 20:15:05,1012.15,0x5c9e30def85334e587cf36eb07bdd6a72bf1452d
2021-05-19 21:26:30,837.05,0x5c9e30def85334e587cf36eb07bdd6a72bf1452d
2021-05-19 19:51:52,591.13,0x5c9e30def85334e587cf36eb07bdd6a72bf1452d
2021-05-19 22:29:39,513.69,0xc64ef1c4c8f417657c6e20b367a6bb4ac4fcb6d3
2021-05-19 07:25:14,301,0x6555e1cc97d3cba6eaddebbcd7ca51d75771e0b8
2021-05-19 00:36:01,299.6,0x4740fa6b32c5b41ebbf631fe1af41e6fff6e2388
2021-05-19 04:28:14,296.3,0x4740fa6b32c5b41ebbf631fe1af41e6fff6e2388
2021-05-19 01:29:37,294.5,0x4740fa6b32c5b41ebbf631fe1af41e6fff6e2388
2021-05-19 05:08:59,291.71,0x4740fa6b32c5b41ebbf631fe1af41e6fff6e2388
